In [1]:
import requests
import re
import urllib 
import os
import xarray as xr

from shapely.geometry import Point
import geopandas as gpd
import numpy as np

from scipy.interpolate import griddata

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def palau_eez(df):
    df_eez = gpd.read_file('./palauEEZ.geojson')
    eez = df_eez['geometry'].values[0]
    in_palau = []
    longlat = df[['longitude','latitude']].values.tolist()
    for x in longlat:
        point = Point(x[0],x[1])
        if eez.contains(point) or point.within(eez):
            in_palau.append((x[0],x[1]))
    return df[df[['longitude', 'latitude']].apply(tuple, axis=1).isin(in_palau)]

In [3]:
def filter_palau(df):
    filtered = df[(df["latitude"]>= 1.25)&
                                (df["latitude"]<=13.00)&(df["longitude"]<=139.00)&
                                (df["longitude"]>=128)]
    return filtered

In [4]:
minlat = -35.0
maxlat = 35.0
minlon = 125.0
maxlon = 360.0-115.0
# date_static = '1986-01-15'
date_static = '1986-01-01'
date_static_t = 'Jan 1986'

In [7]:
conda install -c conda-forge geos

Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized


Note: you may need to restart the kernel to use updated packages.


In [5]:
from cartopy.feature import ShapelyFeature
import cartopy.io.shapereader as shpreader

ImportError: cannot import name lgeos

In [ ]:
# add mapping routines
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
import pandas as pd
import datetime
import os

In [ ]:
shapefile180 = '../../../Misc - Tools/shapefiles/Pacific_Island_EEZ_shifted.shp'
shapefile360 = '../../../Misc - Tools/shapefiles/Pacific_Island_EEZ.shp'

shpf180 = gpd.read_file(shapefile180)
shpf180 = shpf180.loc[:,['COUNTRYNAM','geometry']]
shpf360 = gpd.read_file(shapefile360)
shpf360['COUNTRYNAM'] = shpf180['COUNTRYNAM']
shpf360 = shpf360.loc[:,['COUNTRYNAM','geometry']]

In [ ]:
def convert_to_datetime(dt):
    dt_datetime = datetime.datetime(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second)
    return dt_datetime

In [ ]:
ssp119_list = []
folder = "/Users/one/Documents/Palau/2_ocean/2.2_ocean-temperature/2.2.1_mean-sea-surface-temperature/wgetCMIP6/ssp119/"




for x in os.listdir(folder):
    if "tas_day_EC-Earth3_ssp119_r109i1p1f1_gr_" in x:
        ssp119_list.append(xr.open_dataset(folder + x, decode_times=True))

In [ ]:
def filter_pccm(df):
    filtered = df[(df["latitude"]>= -35)&
                                (df["latitude"]<=35)&(df["longitude"]<=360-115)&
                                (df["longitude"]>=125)]
    return filtered

In [ ]:
ssp119_dfs = []
for x in ssp119_list:
    df = x.tos.to_dataframe()
    df = filter_pccm(df)
    ssp119_dfs.append(df.dropna(subset=['tos']))

In [ ]:
master_ssp119 = pd.concat(ssp119_dfs)
master_ssp119.to_pickle('master_ssp119_pacific.pkl')
master_ssp119.reset_index(inplace = True)

In [ ]:
master_ssp119

In [ ]:
master_ssp119['datetime'] = master_ssp119['time'].apply(convert_to_datetime)

In [ ]:
import pandas as pd
import numpy as np


# Group by latitude and longitude
grouped_master = master_ssp119.groupby(['latitude', 'longitude'])

# Function to calculate trend per coordinate
def calculate_trend(group):
    # Convert datetime to numeric values (timestamps)
    time_numeric = (group['datetime'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
    
    # Extract sst values from the group
    sst = group['tos'].values
    
    # Perform polynomial fit
    poly_coeffs = np.polyfit(time_numeric, sst, deg=1)  # Use degree=1 for linear trend
    
    # Extract the coefficient representing the trend (slope)
    trend_coeff = poly_coeffs[0]
    
    # Convert trend from units per second to units per decade
    trend_deg_per_decade = trend_coeff * 10 * 365.25 * 24 * 60 * 60  # As there are 10 years in a decade
    
    return trend_deg_per_decade

# Apply the function to each group and store the results in a DataFrame
master_ssp119_trend_results = grouped_master.apply(calculate_trend)
master_ssp119_trend_df = master_ssp119_trend_results.reset_index(name='trend')

In [ ]:
master_ssp119_trend_df['long_test'] = master_ssp119_trend_df['longitude'] - 180

In [ ]:
master_ssp119_trend_df

In [ ]:
# Make the figure
fig = plt.figure( figsize = (14, 12) )
vmin, vmax = -0.1, 0.3
divnorm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0., vmax=vmax)

# set projection, center on the Pacific
ax = plt.axes(projection = ccrs.PlateCarree(central_longitude=180.0))


x_orig = np.asarray(master_ssp126_trend_df.long_test.tolist())
y_orig = np.asarray(master_ssp126_trend_df.latitude.tolist())
z_orig = np.asarray(master_ssp126_trend_df.trend.tolist())


x_arr          = np.linspace(np.min(x_orig), np.max(x_orig), 500)
y_arr          = np.linspace(np.min(y_orig), np.max(y_orig), 500)
x_mesh, y_mesh = np.meshgrid(x_arr, y_arr)
z_mesh = griddata((x_orig, y_orig), z_orig, (x_mesh, y_mesh), method='linear')


ctf = plt.contourf(x_mesh, y_mesh,z_mesh, 50,
             vmin=vmin,vmax=vmax, cmap=cm.jet)


    
ax.add_feature(cf.COASTLINE)

# add lat/lon labels (left and bottom)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0, color='gray',
                alpha=0.75, linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False

# set label size and color
gl.xlabel_style = {'size':15, 'color': 'black'}
gl.ylabel_style = {'size':15, 'color': 'black'}

# specify where to label
gl.xlocator = mticker.FixedLocator([140, 160, 180, -160, -140, -120])
gl.ylocator = mticker.FixedLocator([-30, -20, -10, 0, 10, 20, 30])

# specify how to label
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

# add EEZ's as black lines
for i, row in shpf180.iterrows():
    geom = row.geometry
    sp = ShapelyFeature([geom],ccrs.PlateCarree(central_longitude=180.0),
                        edgecolor='black',facecolor='none',lw=0.7)
    ax.add_feature(sp)
    
plt.colorbar(cm.ScalarMappable(norm = ctf.norm, cmap = ctf.cmap),orientation='horizontal',
             label='$^{\circ}C$ per decade')

plt.title('CMIP6 ssp119 tas Trend ')
plt.savefig('projection-sst-ssp119-west-pacific-map.png', format='png',dpi=300,facecolor='white')